# MIT License

Copyright (c) 2024 BullishBanterSociety

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.


In [ ]:
%pip install -r requirements.txt


In [2]:
import yfinance as yf
import pandas as pd
from tabulate import tabulate

def get_financial_data(ticker):
    stock = yf.Ticker(ticker)
    info = stock.info
    financials = stock.financials
    balance_sheet = stock.balance_sheet
    cashflow = stock.cashflow
    return info, financials, balance_sheet, cashflow

# Function to safely get values from DataFrame with a default fallback
def safe_get(df, key, default=0, position=0):
    try:
        value = df.loc[key].iloc[position]
        return value if not pd.isna(value) else default
    except (KeyError, IndexError):
        return default

# Valuation Metrics
def calculate_valuation_metrics(info):
    return {
        'PE Ratio': info.get('forwardPE', 0),  # Price-to-Earnings Ratio: Compares a company's stock price to its earnings per share (EPS). A lower P/E might indicate a stock is undervalued. For instance, a P/E of 10 means the stock price is 10 times the annual earnings per share.
        'PB Ratio': info.get('priceToBook', 0)  # Price-to-Book Ratio: Compares a company's market value to its book value (assets minus liabilities). Useful for assessing if a stock is over or undervalued relative to its assets.
    }

# Profitability Metrics
def calculate_profitability_metrics(financials, balance_sheet, info):
    net_income = safe_get(financials, 'Net Income')
    total_equity = safe_get(balance_sheet, 'Stockholders Equity')
    revenue = safe_get(financials, 'Total Revenue')
    cost_of_goods_sold = safe_get(financials, 'Cost Of Revenue')
    total_assets = safe_get(balance_sheet, 'Total Assets')

    return {
        'ROE': net_income / total_equity if total_equity > 0 else 0,  # Return on Equity: Measures profit generated from shareholders' equity. An ROE of 15% means $0.15 profit for every dollar of equity. High ROE indicates effective management and strong financial performance.
        'Net Profit Margin': net_income / revenue if revenue != 0 else 0,  # Net Profit Margin: Shows profit as a percentage of revenue. A 10% margin means $0.10 profit per dollar of revenue. High net profit margins suggest efficient cost management.
        'Gross Margin': (revenue - cost_of_goods_sold) / revenue if revenue != 0 else 0,  # Gross Margin: Percentage of revenue after accounting for the cost of goods sold. A higher margin indicates better efficiency in production. For example, a 40% gross margin means $0.40 profit per dollar of revenue after production costs.
        'ROA': net_income / total_assets if total_assets > 0 else 0,  # Return on Assets: Efficiency of using assets to generate profit. An ROA of 5% means $0.05 profit per dollar of assets. Higher ROA indicates better asset utilization.
        'EBITDA': safe_get(financials, 'EBITDA'),  # EBITDA: Operational performance without the effects of capital structure, tax rates, or non-cash expenses. Useful for comparing profitability across companies and industries.
    }

# Growth Metrics
def calculate_growth_metrics(financials):
    eps = safe_get(financials, 'Basic EPS', 0)
    previous_eps = safe_get(financials, 'Basic EPS', 0, 1)

    return {
        'EPS Growth': (eps - previous_eps) / previous_eps if previous_eps != 0 else 0  # EPS Growth: Measures growth in earnings per share over time. Consistent growth indicates increasing profitability. For example, a company with EPS growth from $1.00 to $1.10 in a year shows a 10% growth rate.
    }

# Efficiency Metrics
def calculate_efficiency_metrics(financials, balance_sheet):
    revenue = safe_get(financials, 'Total Revenue')
    total_assets = safe_get(balance_sheet, 'Total Assets')
    operating_income = safe_get(financials, 'Operating Income')

    return {
        'Asset Turnover': revenue / total_assets if total_assets > 0 else 0,  # Asset Turnover: Efficiency in using assets to generate sales. An asset turnover of 2 means $2 in sales per dollar of assets. High asset turnover indicates effective asset utilization.
        'Operating Margin': operating_income / revenue if revenue != 0 else 0  # Operating Margin: Profit percentage after paying for production costs. A 15% margin means $0.15 profit per dollar of revenue before interest and taxes. Higher operating margins indicate better operational efficiency.
    }

# Financial Health Metrics
def calculate_financial_health_metrics(balance_sheet, financials):
    total_debt = safe_get(balance_sheet, 'Total Debt')
    total_equity = safe_get(balance_sheet, 'Stockholders Equity')
    current_assets = safe_get(balance_sheet, 'Total Assets')
    current_liabilities = safe_get(balance_sheet, 'Current Liabilities')
    inventory = safe_get(balance_sheet, 'Inventory')
    ebit = safe_get(financials, 'EBIT')
    interest_expense = safe_get(financials, 'Interest Expense')

    return {
        'Debt-to-Equity Ratio': total_debt / total_equity if total_equity > 0 else 0,  # Debt-to-Equity Ratio: Compares total debt to shareholder equity. A lower ratio indicates less reliance on debt, which is typically better. For example, a debt-to-equity ratio of 0.5 means $0.50 of debt for every $1 of equity.
        'Current Ratio': current_assets / current_liabilities if current_liabilities > 0 else 0,  # Current Ratio: Compares current assets to current liabilities. A ratio above 1 indicates more assets than liabilities. A current ratio of 1.5 means $1.50 in current assets for every $1 in current liabilities.
        'Quick Ratio': (current_assets - inventory) / current_liabilities if current_liabilities > 0 else 0,  # Quick Ratio: Similar to the current ratio but excludes inventory for a more conservative view. A quick ratio of 1 or higher suggests the company can meet short-term obligations without selling inventory.
        'Interest Coverage Ratio': ebit / interest_expense if interest_expense != 0 else 0  # Interest Coverage Ratio: Shows how easily a company can pay interest on its debt. A ratio of 3 means operating income is three times interest expenses. Higher ratios indicate better financial health.
    }

# Cash Flow Metrics
def calculate_cashflow_metrics(cashflow, financials):
    free_cash_flow = safe_get(cashflow, 'Free Cash Flow')
    operating_cash_flow = safe_get(cashflow, 'Free Cash Flow')
    net_income = safe_get(financials, 'Net Income')

    return {
        'Free Cash Flow': free_cash_flow,  # Free Cash Flow: Cash left after paying for operating expenses and capital expenditures. Positive and growing FCF indicates a healthy company with the ability to pay dividends, buy back stock, or reinvest in its business.
        'Operating Cash Flow to Net Income': operating_cash_flow / net_income if net_income != 0 else 0  # Operating Cash Flow to Net Income: Compares operating cash flow to net income. A ratio close to or above 1 indicates good earnings quality, suggesting that the company's earnings are backed by actual cash generation.
    }

# Income Metrics
def calculate_income_metrics(info):
    return {
        'Dividend Yield': info.get('dividendYield', 0)  # Dividend Yield: Annual dividend payment divided by stock price, expressed as a percentage. For income-focused investors, a higher dividend yield can be attractive. For example, a stock priced at $100 with an annual dividend of $3 has a dividend yield of 3%. However, very high yields can sometimes indicate potential problems, so consider the company's overall financial health and dividend history.
    }

# Function to calculate all metrics
def calculate_metrics(ticker):
    info, financials, balance_sheet, cashflow = get_financial_data(ticker)
    
    metrics = {}
    metrics.update(calculate_valuation_metrics(info))
    metrics.update(calculate_profitability_metrics(financials, balance_sheet, info))
    metrics.update(calculate_growth_metrics(financials))
    metrics.update(calculate_efficiency_metrics(financials, balance_sheet))
    metrics.update(calculate_financial_health_metrics(balance_sheet, financials))
    metrics.update(calculate_cashflow_metrics(cashflow, financials))
    metrics.update(calculate_income_metrics(info))
    
    return metrics

# Function to get benchmarks
def get_benchmarks():
    return {
        'PE Ratio': 15,
        'PB Ratio': 1.5,
        'ROE': 0.15,
        'EPS Growth': 0.10,
        'Debt-to-Equity Ratio': 0.5,
        'Free Cash Flow': 0,  # Should be positive, no specific benchmark
        'EBITDA': 0,  # No specific benchmark, should be positive
        'EBIT': 0,  # No specific benchmark, should be positive
        'Asset Turnover': 1.0,
        'Operating Margin': 0.15,
        'Dividend Yield': 0.02,  # Example benchmark for dividend yield
        'Current Ratio': 2.0,
        'Quick Ratio': 1.0,
        'Gross Margin': 0.4,
        'Net Profit Margin': 0.1,
        'Interest Coverage Ratio': 3.0,
        'Operating Cash Flow to Net Income': 1.0,
        'ROA': 0.05
    }

# Function to get weights
def get_weights():
    return {
        'PE Ratio': 0.05,
        'PB Ratio': 0.05,
        'ROE': 0.1,
        'EPS Growth': 0.1,
        'Debt-to-Equity Ratio': 0.05,
        'Free Cash Flow': 0.1,
        'EBITDA': 0.05,
        'Asset Turnover': 0.05,
        'Operating Margin': 0.05,
        'Dividend Yield': 0.05,
        'Current Ratio': 0.05,
        'Quick Ratio': 0.05,
        'Gross Margin': 0.05,
        'Net Profit Margin': 0.05,
        'Interest Coverage Ratio': 0.05,
        'Operating Cash Flow to Net Income': 0.05,
        'ROA': 0.05
    }

# Function to score metrics
def score_metrics(metrics):
    benchmarks = get_benchmarks()
    weights = get_weights()
    
    scores = {}
    for metric, value in metrics.items():
        if value is None:
            scores[metric] = 0
            continue
        benchmark = benchmarks.get(metric, 1)
        weight = weights.get(metric, 0.05)
        # Normalize the metric (example normalization, adjust as needed)
        normalized_score = max(0, min(value / benchmark, 1)) if benchmark > 0 else (1 if value > 0 else 0)
        scores[metric] = round(normalized_score * weight * 10, 2)  # Scale to 1-10
    
    total_score = sum(scores.values())
    return scores, round(total_score, 1)

# Function to get explanations
def get_explanations():
    return {
        'PE Ratio': "Price-to-Earnings Ratio: A lower value might indicate the stock is undervalued.",
        'PB Ratio': "Price-to-Book Ratio: A lower value might indicate the stock is undervalued.",
        'ROE': "Return on Equity: Higher values indicate better profitability relative to equity.",
        'EPS Growth': "EPS Growth Rate: Shows the growth rate of earnings per share.",
        'Debt-to-Equity Ratio': "Debt-to-Equity Ratio: Lower values indicate less debt relative to equity.",
        'Free Cash Flow': "Free Cash Flow: The amount of cash available after capital expenditures.",
        'EBITDA': "EBITDA: Earnings before interest, taxes, depreciation, and amortization.",
        'Asset Turnover': "Asset Turnover Ratio: Indicates how efficiently assets generate revenue.",
        'Operating Margin': "Operating Margin: Higher values indicate better operational efficiency.",
        'Dividend Yield': "Dividend Yield: Shows how much a company pays in dividends relative to its stock price.",
        'Current Ratio': "Current Ratio: Indicates liquidity; higher values suggest better ability to meet short-term obligations.",
        'Quick Ratio': "Quick Ratio: Similar to the current ratio but excludes inventory.",
        'Gross Margin': "Gross Margin: Higher values indicate better profitability after accounting for the cost of goods sold.",
        'Net Profit Margin': "Net Profit Margin: Higher values indicate better overall profitability.",
        'Interest Coverage Ratio': "Interest Coverage Ratio: Higher values indicate better ability to pay interest on debt.",
        'Operating Cash Flow to Net Income': "Operating Cash Flow to Net Income Ratio: Indicates the quality of earnings.",
        'ROA': "Return on Assets: Higher values indicate better profitability relative to total assets."
    }

# Function to print metrics and scores with explanations using tabulate
def print_metrics_and_scores(metrics, scores, total_score):
    explanations = get_explanations()
    
    table = []
    for metric, value in metrics.items():
        score = scores.get(metric, 0)
        explanation = explanations.get(metric, "No explanation available.")
        if value is None:
            value = 'N/A'
        table.append([metric, value, score, explanation])
    
    headers = ["Metric", "Value", "Score", "Explanation"]
    print(tabulate(table, headers, tablefmt="grid"))
    print(f"\n{'Total Score':<30} {total_score:<10.1f}")

def get_sp500_tickers():
    sp500_tickers = [
        'AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA', 'META', 'NVDA', 'KO', 'MA',  # Add more tickers as needed
        # ... Add all 500 tickers here
    ]
    return sp500_tickers

# Function to process all S&P 500 companies and sort by score
def process_sp500_companies():
    sp500_tickers = get_sp500_tickers()
    
    results = []
    
    for ticker in sp500_tickers:
        try:
            metrics = calculate_metrics(ticker)
            scores, total_score = score_metrics(metrics)
            results.append((ticker, metrics, scores, total_score))
        except Exception as e:
            print(f"Error processing {ticker}: {e}")
    
    # Sort results by total score in descending order
    results.sort(key=lambda x: x[3], reverse=True)
    
    # Print sorted results
    for ticker, metrics, scores, total_score in results:
        print(f"\n\n{ticker}:")
        print_metrics_and_scores(metrics, scores, total_score)

# Example usage
process_sp500_companies()


/Users/borjaruizdelgadofernandez/Documents/GitHub/gists/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(




KO:
+-----------------------------------+-------------+---------+----------------------------------------------------------------------------------------------------------+
| Metric                            |       Value |   Score | Explanation                                                                                              |
+===================================+=============+=========+==========================================================================================================+
| PE Ratio                          | 22.8812     |    0.5  | Price-to-Earnings Ratio: A lower value might indicate the stock is undervalued.                          |
+-----------------------------------+-------------+---------+----------------------------------------------------------------------------------------------------------+
| PB Ratio                          | 11.555      |    0.5  | Price-to-Book Ratio: A lower value might indicate the stock is undervalued.            